In [30]:
#%pip install pandas

In [31]:
#%pip install numpy

In [32]:
#sklearn
#%pip install scikit-learn

In [33]:
import pandas as pd
import numpy as np

In [34]:
df = pd.read_csv("C:/Lecture Notes/Degree/Sem 8/TCI - Computational Intelligence/new_project/smart-bin-sensor-fill-levels.csv")

In [35]:
df.head()

,bin_id,sensor_id,reading_id,timestamp,fill_level_percent,bin_type,location_latitude,location_longitude,address_street,address_city,address_state,address_postal_code,address_country,battery_level_percent,bin_status,overflow_alert
0,BIN-001,SNSR-1001,READ-00001,2024-06-03T07:34:15Z,12.3,general,40.7128,-74.0060,123 Broadway,New York,NY,10007,USA,83.5,active,False
1,BIN-002,SNSR-1002,READ-00002,2024-06-03T09:21:47Z,69.4,recycling,51.5074,-0.1278,456 Piccadilly,London,England,W1J 9HS,UK,61.2,active,False
2,BIN-003,SNSR-1003,READ-00003,2024-06-03T11:10:32Z,95.1,organic,35.6895,139.6917,789 Shibuya,Tokyo,Tokyo,150-0002,Japan,17.8,maintenance,True
3,BIN-004,SNSR-1004,READ-00004,2024-06-03T13:58:06Z,8.6,hazardous,52.5200,13.4050,23 Alexanderplatz,Berlin,BE,10178,Germany,13.4,out_of_service,False
4,BIN-005,SNSR-1005,READ-00005,2024-06-03T15:46:51Z,46.2,other,-33.8688,151.2093,55 George St,Sydney,NSW,2000,Australia,58.7,active,False


In [36]:
#check for null values
print(df.isnull().sum())

bin_id                   0
sensor_id                0
reading_id               0
timestamp                0
fill_level_percent       0
bin_type                 0
location_latitude        0
location_longitude       0
address_street           3
address_city             0
address_state            0
address_postal_code      0
address_country          0
battery_level_percent    0
bin_status               0
overflow_alert           0
dtype: int64


In [37]:
#drop rows with null values
df = df.dropna()

In [38]:
#convert timestamp to datetime
df['timestamp'] = pd.to_datetime(df['timestamp'], format='mixed', utc=True) 

In [39]:
print(df['timestamp'].head())

0   2024-06-03 07:34:15+00:00
1   2024-06-03 09:21:47+00:00
2   2024-06-03 11:10:32+00:00
3   2024-06-03 13:58:06+00:00
4   2024-06-03 15:46:51+00:00
Name: timestamp, dtype: datetime64[us, UTC]


In [40]:
df = df.sort_values(by=['bin_id', 'timestamp'])


### Feature Engineering

In this part, we will convert categorical values to numerical values, and drop unnecessary columns.
We also combined some columns to create useful entries.

In [41]:
#convert timestamp to hours since last reading for each bin
latest_time = df['timestamp'].max()

df['time_since_last_reading'] = (
    latest_time - df['timestamp']
).dt.total_seconds() / 3600


In [42]:
df.head()

,bin_id,sensor_id,reading_id,timestamp,fill_level_percent,bin_type,location_latitude,location_longitude,address_street,address_city,address_state,address_postal_code,address_country,battery_level_percent,bin_status,overflow_alert,time_since_last_reading
0,BIN-001,SNSR-1001,READ-00001,2024-06-03 07:34:15+00:00,12.3,general,40.7128,-74.0060,123 Broadway,New York,NY,10007,USA,83.5,active,False,71.919444
1,BIN-002,SNSR-1002,READ-00002,2024-06-03 09:21:47+00:00,69.4,recycling,51.5074,-0.1278,456 Piccadilly,London,England,W1J 9HS,UK,61.2,active,False,70.127222
2,BIN-003,SNSR-1003,READ-00003,2024-06-03 11:10:32+00:00,95.1,organic,35.6895,139.6917,789 Shibuya,Tokyo,Tokyo,150-0002,Japan,17.8,maintenance,True,68.314722
3,BIN-004,SNSR-1004,READ-00004,2024-06-03 13:58:06+00:00,8.6,hazardous,52.5200,13.4050,23 Alexanderplatz,Berlin,BE,10178,Germany,13.4,out_of_service,False,65.521944
4,BIN-005,SNSR-1005,READ-00005,2024-06-03 15:46:51+00:00,46.2,other,-33.8688,151.2093,55 George St,Sydney,NSW,2000,Australia,58.7,active,False,63.709444


In [43]:
#normalize 'time_since_last_reading' feature
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
df['time_since_last_reading_norm'] = scaler.fit_transform(
    df[['time_since_last_reading']]
)


In [44]:
df['bin_status'].unique()

<StringArray>
['active', 'maintenance', 'out_of_service']
Length: 3, dtype: str

In [45]:
#encode bin_status
df['bin_status_encoded'] = df['bin_status'].map({'out_of_service': 0, 'active': 1, 'maintenance': 2})

In [46]:
df.head()

,bin_id,sensor_id,reading_id,timestamp,fill_level_percent,bin_type,location_latitude,location_longitude,address_street,address_city,address_state,address_postal_code,address_country,battery_level_percent,bin_status,overflow_alert,time_since_last_reading,time_since_last_reading_norm,bin_status_encoded
0,BIN-001,SNSR-1001,READ-00001,2024-06-03 07:34:15+00:00,12.3,general,40.7128,-74.0060,123 Broadway,New York,NY,10007,USA,83.5,active,False,71.919444,0.166294,1
1,BIN-002,SNSR-1002,READ-00002,2024-06-03 09:21:47+00:00,69.4,recycling,51.5074,-0.1278,456 Piccadilly,London,England,W1J 9HS,UK,61.2,active,False,70.127222,0.162150,1
2,BIN-003,SNSR-1003,READ-00003,2024-06-03 11:10:32+00:00,95.1,organic,35.6895,139.6917,789 Shibuya,Tokyo,Tokyo,150-0002,Japan,17.8,maintenance,True,68.314722,0.157959,2
3,BIN-004,SNSR-1004,READ-00004,2024-06-03 13:58:06+00:00,8.6,hazardous,52.5200,13.4050,23 Alexanderplatz,Berlin,BE,10178,Germany,13.4,out_of_service,False,65.521944,0.151501,0
4,BIN-005,SNSR-1005,READ-00005,2024-06-03 15:46:51+00:00,46.2,other,-33.8688,151.2093,55 George St,Sydney,NSW,2000,Australia,58.7,active,False,63.709444,0.147310,1


In [47]:
df['overflow_alert'].unique()

array([False,  True])

In [48]:
df['overflow_alert'] = df['overflow_alert'].astype(int)
df.head()

,bin_id,sensor_id,reading_id,timestamp,fill_level_percent,bin_type,location_latitude,location_longitude,address_street,address_city,address_state,address_postal_code,address_country,battery_level_percent,bin_status,overflow_alert,time_since_last_reading,time_since_last_reading_norm,bin_status_encoded
0,BIN-001,SNSR-1001,READ-00001,2024-06-03 07:34:15+00:00,12.3,general,40.7128,-74.0060,123 Broadway,New York,NY,10007,USA,83.5,active,0,71.919444,0.166294,1
1,BIN-002,SNSR-1002,READ-00002,2024-06-03 09:21:47+00:00,69.4,recycling,51.5074,-0.1278,456 Piccadilly,London,England,W1J 9HS,UK,61.2,active,0,70.127222,0.162150,1
2,BIN-003,SNSR-1003,READ-00003,2024-06-03 11:10:32+00:00,95.1,organic,35.6895,139.6917,789 Shibuya,Tokyo,Tokyo,150-0002,Japan,17.8,maintenance,1,68.314722,0.157959,2
3,BIN-004,SNSR-1004,READ-00004,2024-06-03 13:58:06+00:00,8.6,hazardous,52.5200,13.4050,23 Alexanderplatz,Berlin,BE,10178,Germany,13.4,out_of_service,0,65.521944,0.151501,0
4,BIN-005,SNSR-1005,READ-00005,2024-06-03 15:46:51+00:00,46.2,other,-33.8688,151.2093,55 George St,Sydney,NSW,2000,Australia,58.7,active,0,63.709444,0.147310,1


In [49]:
df['bin_type'].unique()

<StringArray>
['general', 'recycling', 'organic', 'hazardous', 'other']
Length: 5, dtype: str

In [50]:
#drop unnecessary columns
df = df.drop(columns=['sensor_id', 'reading_id', 'timestamp', 'bin_type', 'location_longitude', 'location_latitude', 'address_street', 'address_city', 'address_state', 'address_postal_code', 'address_country', 'time_since_last_reading', 'bin_status'])

In [51]:
#assign priority labels based on fill_level_percent and overflow_alert
def assign_priority(row):
    if row['overflow_alert'] == 1 or row['fill_level_percent'] >= 85:
        return 2  # High
    elif row['fill_level_percent'] >= 60:
        return 1  # Medium
    else:
        return 0  # Low

df['priority_label'] = df.apply(assign_priority, axis=1)


In [52]:
df.head()

,bin_id,fill_level_percent,battery_level_percent,overflow_alert,time_since_last_reading_norm,bin_status_encoded,priority_label
0,BIN-001,12.3,83.5,0,0.166294,1,0
1,BIN-002,69.4,61.2,0,0.162150,1,1
2,BIN-003,95.1,17.8,1,0.157959,2,2
3,BIN-004,8.6,13.4,0,0.151501,0,0
4,BIN-005,46.2,58.7,0,0.147310,1,0


Finalize features that is used by all models

In [53]:
continuous_features = [
    'fill_level_percent',
    'battery_level_percent',
    'time_since_last_reading_norm'
]

categorical_features = [
    'overflow_alert',
    'bin_status_encoded'
]

bin_number = [
    'bin_id'
]

all_features = continuous_features + categorical_features + bin_number


In [54]:
X = df[all_features]
y = df['priority_label']


In [55]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

X_cont_scaled = pd.DataFrame(
    scaler.fit_transform(X[continuous_features]),
    columns=continuous_features,
    index=X.index
)


In [56]:
X_final = pd.concat(
    [X_cont_scaled, X[categorical_features], X[bin_number]],
    axis=1
)


In [57]:
df.head()

,bin_id,fill_level_percent,battery_level_percent,overflow_alert,time_since_last_reading_norm,bin_status_encoded,priority_label
0,BIN-001,12.3,83.5,0,0.166294,1,0
1,BIN-002,69.4,61.2,0,0.162150,1,1
2,BIN-003,95.1,17.8,1,0.157959,2,2
3,BIN-004,8.6,13.4,0,0.151501,0,0
4,BIN-005,46.2,58.7,0,0.147310,1,0


In [58]:
#convert cleaned data to csv
cleaned_data = pd.concat([X_final, y], axis=1)
cleaned_data.to_csv("cleaned_smart_bin_data.csv", index=False)